<a href="https://colab.research.google.com/github/nthuTaiwan/Prediction-procedure/blob/main/iGEM_ProteinMPNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Block 1. 基礎設定：連接雲端硬碟與啟用 GPU
---
這個區塊負責所有初始設定，每次重啟會話時都需要執行。

In [ ]:
# @title 1.1 驗證GPU環境
!nvidia-smi

Thu Jun 26 15:43:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 連線 Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title 1.2 設定專案路徑
import os
PROJECT_PATH = '/content/drive/MyDrive/iGEM_ProteinMPNN'
os.makedirs(PROJECT_PATH, exist_ok=True)
%cd {PROJECT_PATH}

/content/drive/MyDrive/iGEM_ProteinMPNN


Block 2. 安裝所有軟體
---
只需執行一次，如果已經安裝過，可以跳過此區塊。

In [ ]:
# @title 2.1 安裝ProteinMPNN
!git clone https://github.com/dauparas/ProteinMPNN.git
%cd ProteinMPNN
!pip install torch torchvision torchaudio
!pip install numpy

Cloning into 'ProteinMPNN'...
remote: Enumerating objects: 634, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 634 (delta 185), reused 154 (delta 154), pack-reused 395 (from 1)
Receiving objects: 100% (634/634), 119.91 MiB | 16.73 MiB/s, done.
Resolving deltas: 100% (285/285), done.
Updating files: 100% (109/109), done.
/content/drive/MyDrive/iGEM/ProteinMPNN/ProteinMPNN/ProteinMPNN


In [ ]:
# @title 2.2 安裝 GROMACS
!pip install -q condacolab
import condacolab
condacolab.install()
!mamba install -y -c conda-forge gromacs

✨🍰✨ Everything looks OK!

Looking for: ['gromacs']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



Block 3. 執行工作流程
---
從這裡開始是主要的研究循環。

In [ ]:
# @title 3.0.1 設置 bias_aa_jsonl
# /ProteinMPNN/helper_scripts/ Folder裡面還有很多helper scripts
!python /content/drive/MyDrive/iGEM_ProteinMPNN/ProteinMPNN/helper_scripts/make_bias_AA.py \
    --output_path bias_AA.jsonl \
    --AA_list "Y" \
    --bias_list "100.0"

In [ ]:
!python test.py \
       --input_path /content/drive/MyDrive/iGEM_ProteinMPNN/inputs/backbones \
       --output_path parsed_pdbs.jsonl

Found 6 PDB files to process.
Processing file 1/6: test_87fb1_unrelaxed_rank_002_alphafold2_ptm_model_5_seed_000.pdb
Processing file 2/6: test_87fb1_unrelaxed_rank_005_alphafold2_ptm_model_4_seed_000.pdb
Processing file 3/6: test_87fb1_unrelaxed_rank_004_alphafold2_ptm_model_1_seed_000.pdb
Processing file 4/6: test_87fb1_unrelaxed_rank_003_alphafold2_ptm_model_3_seed_000.pdb
Processing file 5/6: test_87fb1_unrelaxed_rank_001_alphafold2_ptm_model_2_seed_000.pdb
Processing file 6/6: SopE.pdb

Processing finished. Saved 6 parsed structures to parsed_pdbs.jsonl


In [ ]:
# @title 3.0.2 設置: parse_multiple_chains.py
# /ProteinMPNN/helper_scripts/ Folder裡面還有很多helper scripts
!python ProteinMPNN/helper_scripts/parse_multiple_chains.py \
       --input_path inputs/backbones \
       --output_path parsed_pdbs.jsonl

In [ ]:
!python ProteinMPNN/helper_scripts/make_fixed_positions_dict.py \
       --input_path parsed_pdbs.jsonl \
       --output_path fixed_positions.jsonl \
       --chain_list "A" \
       --position_list "$(seq 1 76 | tr '\n' ' ')"

In [ ]:
# @title 3.1 步驟 A：【自動化批次執行】ProteinMPNN 生成序列
# 會自動尋找 backbones 資料夾中所有的 .pdb files，並為每一個檔案執行 ProteinMPNN。

import os
try:
    PROJECT_PATH
except NameError:
    PROJECT_PATH = '/content/drive/MyDrive/iGEM_ProteinMPNN'

backbones_folder = f"{PROJECT_PATH}/inputs/backbones"
output_folder = f"{PROJECT_PATH}/outputs/mpnn_sequences/"
bias_aa_path = f"{PROJECT_PATH}/bias_AA.jsonl"
fixed_positions_path = f"{PROJECT_PATH}/fixed_positions.jsonl"
os.makedirs(backbones_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

try:
    pdb_files = [f for f in os.listdir(backbones_folder) if f.endswith('.pdb')]
    if not pdb_files:
        print(f"警告：在 '{backbones_folder}' 中沒有找到任何 .pdb 檔案。")
    else:
        print(f"成功找到 {len(pdb_files)} 個 PDB 檔案，準備處理：")
        print(pdb_files)
except FileNotFoundError:
    print(f"發生錯誤")
    pdb_files = [] # 建立一個空列表以避免後續錯誤

if pdb_files:
    %cd {PROJECT_PATH}/ProteinMPNN
    for pdb_file_name in pdb_files:
        current_pdb_input_path = os.path.join(backbones_folder, pdb_file_name)

        print("\n" + "="*50)
        print(f"正在處理檔案： {pdb_file_name}")
        print(f"完整輸入路徑： {current_pdb_input_path}")
        print("="*50 + "\n")

        !python protein_mpnn_run.py \
                --pdb_path "{current_pdb_input_path}" \
                --out_folder "{output_folder}" \
                --num_seq_per_target 10 \
                --bias_AA_jsonl "{bias_aa_path}" \
                --fixed_positions_jsonl "{fixed_positions_path}"
print("\n所有檔案處理完畢！")
%cd {PROJECT_PATH}

成功找到 6 個 PDB 檔案，準備處理：
['test_87fb1_unrelaxed_rank_002_alphafold2_ptm_model_5_seed_000.pdb', 'test_87fb1_unrelaxed_rank_005_alphafold2_ptm_model_4_seed_000.pdb', 'test_87fb1_unrelaxed_rank_004_alphafold2_ptm_model_1_seed_000.pdb', 'test_87fb1_unrelaxed_rank_003_alphafold2_ptm_model_3_seed_000.pdb', 'test_87fb1_unrelaxed_rank_001_alphafold2_ptm_model_2_seed_000.pdb', 'SopE.pdb']
/content/drive/MyDrive/iGEM_ProteinMPNN/ProteinMPNN

正在處理檔案： test_87fb1_unrelaxed_rank_002_alphafold2_ptm_model_5_seed_000.pdb
完整輸入路徑： /content/drive/MyDrive/iGEM_ProteinMPNN/inputs/backbones/test_87fb1_unrelaxed_rank_002_alphafold2_ptm_model_5_seed_000.pdb

----------------------------------------
chain_id_jsonl is NOT loaded
----------------------------------------
pssm_jsonl is NOT loaded
----------------------------------------
omit_AA_jsonl is NOT loaded
----------------------------------------
tied_positions_jsonl is NOT loaded
----------------------------------------
bias by residue dictionary is not loade

In [ ]:
# @title 3.2 步驟 B：全自動執行 ColabFold 結構預測 (Still building)
# 這個儲存格會自動完成以下任務：
# 1. 安裝 ColabFold 指令行工具。
# 2. 讀取 ProteinMPNN 生成的多序列 FASTA 檔案。
# 3. 對檔案中的 **每一個序列** 執行 AlphaFold2 預測。
# 4. 將每個預測結果（PDB、圖片等）分別儲存在以序列名命名的資料夾中。

import os
print("安裝 ColabFold 中... (可能需要幾分鐘)")
!pip install -q "colabfold[alphafold] @ git+https://github.com/sokrypton/ColabFold"
print("ColabFold 安裝完成！")

MPNN_OUTPUT_DIR = f"{PROJECT_PATH}/outputs/mpnn_sequences/seqs/"
COLABFOLD_RESULTS_DIR = f"{PROJECT_PATH}/outputs/colabfold_results/"
os.makedirs(COLABFOLD_RESULTS_DIR, exist_ok=True)

FASTA_INPUT_FILE = os.path.join(MPNN_OUTPUT_DIR, "test_87fb1_unrelaxed_rank_001_alphafold2_ptm_model_2_seed_000.fa")

os.environ['MPLBACKEND'] = 'agg'
print(f"Matplotlib backend 已設定為 'agg' 來避免衝突。")

print(f"開始對檔案 '{os.path.basename(FASTA_INPUT_FILE)}' 中的所有序列進行預測...")
!colabfold_batch {FASTA_INPUT_FILE} {COLABFOLD_RESULTS_DIR}

print("="*50)
print(f"所有序列預測完成！")
print(f"結果已儲存至: {COLABFOLD_RESULTS_DIR}")
print("="*50)

安裝 ColabFold 中... (可能需要幾分鐘)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorf